In [ ]:
# Install necessary libraries
!pip install requests pandas transformers ipywidgets torch plotly sentence-transformers datasets accelerate bitsandbytes

# Enable ipywidgets for Google Colab
from google.colab import output
output.enable_custom_widget_manager()

#API Key KA1DC5SYOT5WUAIL 313d44edfdad48f9bacdfb1d2441ea82

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinu

In [ ]:
import requests
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer, default_data_collator, EarlyStoppingCallback, BartTokenizer, BartForConditionalGeneration
import torch
import ipywidgets as widgets
from IPython.display import display, HTML
import re
import plotly.graph_objects as go
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# Fubction to fetch intraday stock data
def get_stock_data(api_key, symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    if 'Time Series (1min)' in data:
        df = pd.DataFrame(data['Time Series (1min)']).T
        df = df.rename(columns={
            '1. open': 'open',
            '2. high': 'high',
            '3. low': 'low',
            '4. close': 'close',
            '5. volume': 'volume'
        })
        df.index = pd.to_datetime(df.index)
        return df
    else:
        raise ValueError(f"Data not available or API request failed. Response: {data}")

In [ ]:
# Function to fetch financial news
def get_latest_news(symbol, api_key):
    news_url = f'https://newsapi.org/v2/everything?q={symbol}&apiKey={api_key}'
    response = requests.get(news_url)
    return response.json()

In [ ]:
# Clean text function
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    text = re.sub(r'[^A-Za-z0-9\s]+', '', text)  # Remove special characters
    text = text.strip()
    return text

In [ ]:
# Load models
finbert_tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finbert_model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
sentence_encoder = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
# Function embeeding
def get_embeddings(text):
    embeddings = sentence_encoder.encode(text)
    return embeddings

In [ ]:
# Create Custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
class MetricsCalculator:
    def __init__(self):
        self.metric = load_metric("accuracy")
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def compute_metrics(self, eval_pred):
        predictions = eval_pred.predictions
        labels = eval_pred.label_ids

        # Calculate accuracy
        accuracy = self.metric.compute(predictions=predictions.argmax(axis=1), references=labels)

        # Calculate loss
        loss = self.loss_fn(torch.tensor(predictions), torch.tensor(labels)).item()

        # Calculate precision, recall, f1-score
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions.argmax(axis=1), average='weighted')

        return {
            'eval_accuracy': accuracy['accuracy'],
            'eval_loss': loss,
            'eval_precision': precision,
            'eval_recall': recall,
            'eval_f1': f1,
        }

In [ ]:
# Function to fine-tune a pre-trained model
def custom_fine_tune(transformer_model, tokenizer, train_dataset, val_dataset, output_dir, epochs=3, batch_size=16, learning_rate=2e-5):
    training_args = TrainingArguments(
        output_dir=output_dir,  # Specify the output directory for saving results
        per_device_train_batch_size=batch_size,
        num_train_epochs=epochs,
        per_device_eval_batch_size=batch_size,
        eval_strategy='epoch',  # Updated according to warning
        save_strategy='epoch',
        logging_dir='./logs',
        fp16=True,
        gradient_accumulation_steps=2,
        learning_rate=learning_rate,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        logging_steps=100,
        weight_decay=0.01,
    )

    num_labels = 2
    model = AutoModelForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=num_labels,
        hidden_dropout_prob=0.4,
        attention_probs_dropout_prob=0.4
    )

    # Instantiate your MetricsCalculator
    metrics_calculator = MetricsCalculator()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=metrics_calculator.compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    # Start training
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()

    # Save the model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    return eval_results

In [ ]:
# Function to generate advice using BART
def generate_advice(prompt):
    inputs = bart_tokenizer.encode("summarize: " + prompt, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = bart_model.generate(inputs, max_length=512, min_length=200, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# Function to summarize text using BART
def summarize_text(text):
    inputs = bart_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = bart_model.generate(inputs, max_length=300, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# Function for stock analysis before fine-tuning
def analyze_stock_before_finetune(alpha_vantage_api_key, news_api_key, symbol):
    stock_data = get_stock_data(alpha_vantage_api_key, symbol)
    latest_price = stock_data['close'][0]

    # Fetch latest news
    news_data = get_latest_news(symbol, news_api_key)
    news_summary_cleaned = []

    if news_data and 'articles' in news_data:
        for article in news_data['articles'][:5]:
            description = clean_text(article['description'])
            inputs = finbert_tokenizer(description, return_tensors='pt', padding=True, truncation=True, max_length=512)
            outputs = finbert_model(**inputs)
            sentiment = torch.argmax(outputs.logits, dim=1).item()
            sentiment_label = "Positive" if sentiment == 1 else "Negative" if sentiment == 0 else "Neutral"

            # Summarize news using finbert model
            summary = summarize_text(description)

            # Get embeddings of the cleaned text
            embeddings = get_embeddings(description)

            news_summary_cleaned.append((article['title'], sentiment_label, summary, embeddings))

    # Investment advice with detailed prompt
    advice_prompt = (
        f"Current market trends show a bullish movement in the tech sector. {symbol} has been gaining momentum. "
        f"News articles suggest significant investor interest. Given the current price of {symbol}, generate detailed investment advice "
        f"considering market trends, stock performance, and financial news. "
        f"Provide a thorough analysis including potential risks, market conditions, and long-term investment potential. "
        f"Discuss the stock's historical performance, recent news impact, and any upcoming events that may influence its price. "
        f"Also, offer a strategic plan for both short-term and long-term investors."
    )

    advice = generate_advice(advice_prompt)

    return latest_price, advice, news_summary_cleaned, stock_data

In [ ]:
# Create UI before fine-tuning
alpha_vantage_api_key_widget = widgets.Text(value='', placeholder='Enter your Alpha Vantage API Key', description='Alpha Vantage API Key:')
news_api_key_widget = widgets.Text(value='', placeholder='Enter your News API Key', description='News API Key:')
symbol_widget = widgets.Text(value='', placeholder='Enter stock symbol (e.g., AAPL)', description='Stock Symbol:')
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        alpha_vantage_api_key = alpha_vantage_api_key_widget.value
        news_api_key = news_api_key_widget.value
        symbol = symbol_widget.value
        if alpha_vantage_api_key and news_api_key and symbol:
            latest_price, advice, news_summary_cleaned, stock_data = analyze_stock_before_finetune(alpha_vantage_api_key, news_api_key, symbol)
            display(HTML(f'<h3>Current Price of {symbol}: {latest_price}</h3>'))
            display(HTML(f'<h4>Investment Advice:</h4><p>{advice}</p>'))
            display(HTML('<h4>Latest Financial News:</h4>'))
            for title, sentiment, summary, embeddings in news_summary_cleaned:
                #display(HTML(f"<b>{title}</b> (Sentiment: {sentiment})<br>{summary}<br>Embeddings: {embeddings}<br><br>"))
                display(HTML(f"<b>{title}</b> (Sentiment: {sentiment})<br>{summary}<br><br>"))

            # Visualize stock data
            fig = go.Figure(data=[go.Candlestick(
                x=stock_data.index,
                open=stock_data['open'],
                high=stock_data['high'],
                low=stock_data['low'],
                close=stock_data['close']
            )])
            fig.update_layout(title=f'Stock Price Data for {symbol}', yaxis_title='Price (USD)', xaxis_title='Time')
            fig.show()
        else:
            print("Please enter the Alpha Vantage API key, News API key and stock symbol.")

button = widgets.Button(description="Analyze Stock")
button.on_click(on_button_click)

In [ ]:
#API Key KA1DC5SYOT5WUAIL313d44edfdad48f9bacdfb1d2441ea82

In [ ]:
# Display UI before fine-tuning
display(alpha_vantage_api_key_widget, news_api_key_widget, symbol_widget, button, output)

Text(value='', description='Alpha Vantage API Key:', placeholder='Enter your Alpha Vantage API Key')

Text(value='', description='News API Key:', placeholder='Enter your News API Key')

Text(value='', description='Stock Symbol:', placeholder='Enter stock symbol (e.g., AAPL)')

Button(description='Analyze Stock', style=ButtonStyle())

Output()

In [ ]:
# Fine-tuning example (run separately)
# Load datasets
dataset = load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Split into training and validation sets
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
val_texts = dataset['test']['text']
val_labels = dataset['test']['label']

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Create custom datasets
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length=128)

In [ ]:
# Load the pre-trained model
pretrained_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Evaluate the pre-trained model before fine-tuning

# Instantiate your MetricsCalculator
metrics_calculator = MetricsCalculator()

# Evaluate the pre-trained model before fine-tuning
training_args = TrainingArguments(
    output_dir='./results_before',
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    fp16=True,
    gradient_accumulation_steps=2,
)

trainer_before = Trainer(
    model=pretrained_model,
    args=training_args,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=metrics_calculator.compute_metrics
)

eval_results_before = trainer_before.evaluate()

<ipython-input-9-9b51b44c6364>:3: FutureWarning:

load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate



The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
# Print evaluation results before fine-tuning
print("Evaluation results before fine-tuning:")
print(f"Loss: {eval_results_before['eval_loss']}")
print(f"Accuracy: {eval_results_before['eval_accuracy']}")
print(f"Precision: {eval_results_before['eval_precision']}")
print(f"Recall: {eval_results_before['eval_recall']}")
print(f"F1-score: {eval_results_before['eval_f1']}")

Evaluation results before fine-tuning:
Loss: 0.6966903805732727
Accuracy: 0.4936
Precision: 0.45833333333333326
Recall: 0.4936
F1-score: 0.3576864535768645


In [ ]:
# Fine-tune the pre-trained model with 20 epoch
eval_results_after = custom_fine_tune(pretrained_model, tokenizer, train_dataset, val_dataset, output_dir='./results_after_finetune', epochs=20)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.392200,0.325202,0.866160,0.866756,0.866160,0.866106
2,0.309700,0.314706,0.877800,0.878828,0.877800,0.877717
4,0.258400,0.380070,0.879760,0.882606,0.879760,0.879536
6,0.228900,0.312147,0.886360,0.887101,0.886360,0.886306
8,0.195800,0.334379,0.884080,0.885652,0.884080,0.883962
10,0.183300,0.358593,0.885960,0.885967,0.885960,0.885959


In [ ]:
# Print evaluation results after fine-tuning
print("Evaluation results after fine-tuning:")
print(f"Loss: {eval_results_after['eval_loss']}")
print(f"Accuracy: {eval_results_after['eval_accuracy']}")
print(f"Precision: {eval_results_after['eval_precision']}")
print(f"Recall: {eval_results_after['eval_recall']}")
print(f"F1-score: {eval_results_after['eval_f1']}")

Evaluation results after fine-tuning:
Loss: 0.31214722990989685
Accuracy: 0.88636
Precision: 0.8871012747860959
Recall: 0.88636
F1-score: 0.8863055705655413


In [ ]:
# Load the fine-tuning model
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained('./results_after_finetune')
fine_tuned_tokenizer = AutoTokenizer.from_pretrained('./results_after_finetune')

In [ ]:
# Save model after fine-tuning
fine_tuned_model.save_pretrained('./saved_model')
fine_tuned_tokenizer.save_pretrained('./saved_model')

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
# Function for stock analysis after fine-tuning
def analyze_stock_after_finetune(alpha_vantage_api_key, news_api_key, symbol):
    stock_data = get_stock_data(alpha_vantage_api_key, symbol)
    latest_price = stock_data['close'].iloc[0]

    news_data = get_latest_news(symbol, news_api_key)
    news_summary_cleaned = []

    if news_data and 'articles' in news_data:
        for article in news_data['articles'][:5]:
            description = clean_text(article['description'])
            inputs = fine_tuned_tokenizer(description, return_tensors='pt', padding=True, truncation=True, max_length=512)
            outputs = fine_tuned_model(**inputs)
            sentiment = torch.argmax(outputs.logits, dim=1).item()
            sentiment_label = "Positive" if sentiment == 1 else "Negative" if sentiment == 0 else "Neutral"

            news_summary_cleaned.append((article['title'], sentiment_label, description))

    advice_prompt = (
        f"Current market trends show a bullish movement in the tech sector. {symbol} has been gaining momentum. "
        f"News articles suggest significant investor interest. Given the current price of {symbol}, generate detailed investment advice "
        f"considering market trends, stock performance, and financial news. "
        f"Provide a thorough analysis including potential risks, market conditions, and long-term investment potential. "
        f"Discuss the stock's historical performance, recent news impact, and any upcoming events that may influence its price. "
        f"Also, offer a strategic plan for both short-term and long-term investors."
    )

    advice = generate_advice(advice_prompt)
    return latest_price, advice, news_summary_cleaned

In [ ]:
# Create UI after fine-tuning
alpha_vantage_api_key_widget = widgets.Text(value='', placeholder='Enter your Alpha Vantage API Key', description='Alpha Vantage API Key:')
news_api_key_widget = widgets.Text(value='', placeholder='Enter your News API Key', description='News API Key:')
symbol_widget = widgets.Text(value='', placeholder='Enter stock symbol (e.g., AAPL)', description='Stock Symbol:')
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        alpha_vantage_api_key = alpha_vantage_api_key_widget.value
        news_api_key = news_api_key_widget.value
        symbol = symbol_widget.value
        if alpha_vantage_api_key and news_api_key and symbol:
            try:
                latest_price, advice, news_summary_cleaned = analyze_stock_after_finetune(alpha_vantage_api_key, news_api_key, symbol)

                display(HTML(f'<h3>Current Price of {symbol}: {latest_price}</h3>'))
                display(HTML(f'<h4>Investment Advice:</h4><p>{advice}</p>'))
                display(HTML('<h4>Latest Financial News:</h4>'))

                for title, sentiment, summary in news_summary_cleaned:
                    display(HTML(f"<b>{title}</b> (Sentiment: {sentiment})<br>{summary}<br><br>"))

                # Visualize stock data separately
                stock_data = get_stock_data(alpha_vantage_api_key, symbol)
                fig = go.Figure(data=[go.Candlestick(
                    x=stock_data.index,
                    open=stock_data['open'],
                    high=stock_data['high'],
                    low=stock_data['low'],
                    close=stock_data['close']
                )])
                fig.update_layout(title=f'Stock Price Data for {symbol}', yaxis_title='Price (USD)', xaxis_title='Time')
                fig.show()
            except Exception as e:
                print(f"An error occurred: {e}")
        else:
            print("Please enter the Alpha Vantage API key, News API key, and stock symbol.")

button = widgets.Button(description="Analyze Stock")
button.on_click(on_button_click)

In [ ]:
#API Key KA1DC5SYOT5WUAIL313d44edfdad48f9bacdfb1d2441ea82

In [ ]:
# Display UI after fine-tuning
display(alpha_vantage_api_key_widget, news_api_key_widget, symbol_widget, button, output)

In [ ]:
# Load saved model
saved_model_path = './saved_model'
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(saved_model_path)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(saved_model_path)